In [1]:
import os

In [2]:
import importlib

In [3]:
from pyiron_base import state

In [4]:
from pyiron_atomistics import Project

In [5]:
from pyiron_atomistics.lammps.interactive import LammpsInteractive

In [6]:
class LammpsInteractiveWithoutOutput(LammpsInteractive):
    def __init__(self, project, job_name):
        super(LammpsInteractiveWithoutOutput, self).__init__(project, job_name)
        self._interactive_disable_log_file = False
    
    def to_hdf(self, hdf=None, group_name=None):
        """

        Args:
            hdf:
            group_name:

        Returns:

        """
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput, self).to_hdf(hdf=hdf, group_name=group_name)
    
    def interactive_flush(self, path="interactive", include_last_step=False):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput, self).interactive_flush(path=path, include_last_step=include_last_step)

    def interactive_initialize_interface(self):
        if not self._interactive_disable_log_file:
            self._create_working_directory()
        if self.server.run_mode.interactive and self.server.cores == 1:
            lammps = getattr(importlib.import_module("lammps"), "lammps")
            if self._log_file is None:
                self._log_file = os.path.join(self.working_directory, "log.lammps")
            if not self._interactive_disable_log_file:
                self._interactive_library = lammps(
                    cmdargs=["-screen", "none", "-log", self._log_file]
                )
            else:
                self._interactive_library = lammps(
                    cmdargs=["-screen", "none", "-log", "none"]
                )
        else:
            self._interactive_library = LammpsLibrary(
                cores=self.server.cores, working_directory=self.working_directory
            )
        if not all(self.structure.pbc):
            self.input.control["boundary"] = " ".join(
                ["p" if coord else "f" for coord in self.structure.pbc]
            )
        self._reset_interactive_run_command()
        self.interactive_structure_setter(self.structure)
        
    def interactive_close(self):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput).interactive_close()
            
    def refresh_job_status(self):
        if not self._interactive_disable_log_file:
            super(LammpsInteractiveWithoutOutput).refresh_job_status()

In [7]:
state.settings.configuration

{'user': 'pyiron',
 'resource_paths': ['/Users/janssen/pyiron/resources',
  '/Users/janssen/mambaforge/share/pyiron'],
 'project_paths': [],
 'connection_timeout': 60,
 'sql_connection_string': None,
 'sql_table_name': 'jobs_pyiron',
 'sql_view_connection_string': None,
 'sql_view_table_name': None,
 'sql_view_user': None,
 'sql_view_user_key': None,
 'sql_file': '/Users/janssen/pyiron.db',
 'sql_host': None,
 'sql_type': 'SQLite',
 'sql_user_key': None,
 'sql_database': None,
 'project_check_enabled': False,
 'disable_database': True,
 'credentials_file': None,
 'write_work_dir_warnings': True,
 'config_file_permissions_warning': True}

In [8]:
project = Project("elastic")

In [9]:
project.remove_jobs(recursive=True, silently=True)

In [10]:
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [11]:
structure = project.create.structure.ase.bulk("Fe", cubic=True)
structure

Fe: [0. 0. 0.]
Fe: [1.435 1.435 1.435]
pbc: [ True  True  True]
cell: 
Cell([2.87, 2.87, 2.87])

In [12]:
lmp_mini1 = project.create_job(LammpsInteractiveWithoutOutput, "lmp_mini", delete_existing_job=True)
lmp_mini1.structure = structure
lmp_mini1.potential = potential
lmp_mini1.calc_minimize(pressure=0.0)
lmp_mini1.server.run_mode.interactive = True
lmp_mini1._interactive_disable_log_file = True  # disable lammps.log 
lmp_mini1.run()
lmp_mini1.interactive_close()

The job lmp_mini was saved and received the ID: lmp_mini


/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/PycharmProjects/pyiron_atomistics/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [13]:
lmp_mini1.get_structure()

Fe: [0.00482899 0.00482899 0.00482899]
Fe: [1.435 1.435 1.435]
pbc: [ True  True  True]
cell: 
Cell([[2.8603420110170252, 1.7514543441293517e-16, 1.7514543441293517e-16], [0.0, 2.8603420110170252, 1.7514543441293512e-16], [0.0, 0.0, 2.8603420110170252]])